<a href="https://colab.research.google.com/github/Asato4931/CA_5_2/blob/main/5_2_Fine_Tuning_%2B_STS_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer

from sentence_transformers import SentenceTransformer
from sentence_transformers import models
 
import numpy as np

import pandas as pd

from scipy import stats

from sentence_transformers.losses import TripletDistanceMetric, TripletLoss
from sentence_transformers.evaluation import TripletEvaluator
from sentence_transformers.readers import TripletReader
from sentence_transformers.datasets import SentencesDataset
from torch.utils.data import DataLoader


from sentence_transformers.losses import InputExample, losses





# Load pre-trained tokenizer
transformer = models.Transformer("cl-tohoku/bert-base-japanese-v2")
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v2")
#model = AutoModelForMaskedLM.from_pretrained("cl-tohoku/bert-base-japanese-v2")
pooling = models.Pooling(transformer.get_word_embedding_dimension(), pooling_mode_mean_tokens=True, pooling_mode_cls_token=False, pooling_mode_max_tokens=False)
model = SentenceTransformer(modules=[transformer, pooling])



train_data = pd.read_json('/content/drive/MyDrive/Colab Notebooks/CA課題用/CA課題5/train-v1.1.json', lines=True)


train_data = train_data.loc[:,['sentence1','sentence2']]
train_label = train_data.loc[:,['label']]
train_label = train_label / 5

pearson_train = []



#Pearson

for i in range(0,10):
  vector1_pt = model.encode(train_data.loc[i,'sentence1'])
  vector2_pt = model.encode(train_data.loc[i,'sentence2'])

  result_pt= stats.pearsonr(vector1_pt,vector2_pt)
  pearson_train.append(result_pt.statistic)


pearson_train_df = pd.DataFrame(pearson_train)
pearson_train_df["metrics"] = "pearson"
pearson_train_df = pearson_train_df.rename(columns= { pearson_train_df.columns[0]: "score"})
pearson_train_df = pearson_train_df.reindex(columns=['metrics', 'score'])



BATCH_SIZE = 16
NUM_EPOCHS = 1
EVAL_STEPS = 1000
WARMUP_STEPS = int(len(train_data) // BATCH_SIZE * 0.1) 
#OUTPUT_PATH = "./sbert_stair"



triplet_reader = TripletReader(".")
train_data = SentencesDataset(triplet_reader.get_examples('triplet_train.tsv'), model=model)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)



train_loss = CosineSimilarityLoss(model=model)

dev_data = SentencesDataset(triplet_reader.get_examples('triplet_dev.tsv'), model=model)
dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=BATCH_SIZE)
evaluator = TripletEvaluator(dev_dataloader)

model.fit(train_objectives=[(train_dataloader, train_loss)],evaluator=evaluator,epochs=NUM_EPOCHS,evaluation_steps=EVAL_STEPS,warmup_steps=WARMUP_STEPS)




valid_data = pd.read_json('/content/drive/MyDrive/Colab Notebooks/CA課題用/CA課題5/valid-v1.1.json', lines=True)


valid_data = valid_data.loc[:,['sentence1','sentence2']]



pearson_val = []
spearman_val = []



#Pearson

for i in range(0,1457):
  vector1_pv = model.encode(valid_data.loc[i,'sentence1'])
  vector2_pv = model.encode(valid_data.loc[i,'sentence2'])

  result_pv= stats.pearsonr(vector1_pv,vector2_pv)
  pearson_val.append(result_pv.statistic)


#Spearman

for i in range(0,1457):
  vector1_s = model.encode(valid_data.loc[i,'sentence1'])
  vector2_s = model.encode(valid_data.loc[i,'sentence2'])

  vector1_s = np.array(vector1_s).reshape(1,-1)
  vector2_s = np.array(vector2_s).reshape(1,-1)
  result_s = stats.spearmanr(vector1_s,vector2_s ,axis=None)
  spearman_val.append(result_s.statistic)




pearson_val_df = pd.DataFrame(pearson_val)
pearson_val_df = pearson_val_df * 5
pearson_val_df["metrics"] = "pearson"
pearson_val_df = pearson_val_df.rename(columns= { pearson_val_df.columns[0]: "score"})
pearson_val_df = pearson_val_df.reindex(columns=['metrics', 'score'])




spearman_val_df = pd.DataFrame(spearman_val)
spearman_val_df = spearman_val_df * 5
spearman_val_df["metrics"] = "spearman"
spearman_val_df = spearman_val_df.rename(columns= { spearman_val_df.columns[0]: "score"})
spearman_val_df = spearman_val_df.reindex(columns=['metrics', 'score'])


frames = [pearson_val_df, spearman_val_df]

result_5_2 = pd.concat(frames)

result_5_2.to_json('/content/drive/MyDrive/Colab Notebooks/CA課題用/CA課題5/CA_5_2 STS Task Results.jsonl', force_ascii=False, lines = True, orient='records' )



In [ ]:
!pip install -U sentence-transformers

!apt-get install mecab mecab-ipadic-utf8 python-mecab libmecab-dev
!pip install mecab-python3 

!pip install fugashi

!pip install unidic_lite